In [75]:
import pandas as pd
import numpy as np
import tensorflow as tf

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [77]:
def load_mnist_csv(path = "/data/MNIST/", one_hot = False, shape = None):
    df_train = pd.read_csv(path + "mnist_train.csv", header=None)
    df_test = pd.read_csv(path + "mnist_test.csv", header=None)
    
    X_train = df_train.iloc[:, 1:].values/255
    X_test = df_test.iloc[:, 1:].values/255
    y_train = df_train.iloc[:, 0].values
    y_test = df_test.iloc[:, 0].values
    
    if shape == "2D":
        X_train = X_train.reshape(-1, 28, 28)
        X_test = X_test.reshape(-1, 28, 28)
        
    if shape == "3D":
        X_train = X_train.reshape(-1, 28, 28, 1)
        X_test = X_test.reshape(-1, 28, 28, 1)
    
    if one_hot:
        eye = np.eye(len(np.unique(y_train)))
        y_train, y_test = eye[y_train], eye[y_test]
        
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_mnist_csv(shape="2D")

In [147]:
class BatchableInfo:
    pass

class Batchable(object):
    """
    Create a batchable object that would return batched X and y values.
    
    Usage:
    
    init = tf.global_variables_initializer()
    batchable = Batchable(X_train, y_train, epochs=100)
    with tf.Session() as sess:
        init.run()
        for X_batch, y_batch, info in batchable.next():
            sess.run(opt, feed_dict={X: X_batch, y: y_batch})
            if info.show_progress:
                acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
                acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
                print("Progress:%3d%%" % info.progress, 
                      "Train accuracy: %.4f" % acc_train, 
                      "Test accuracy: %.4f (ms)" % acc_test,
                      "Rate %.4f" % info.rate)
    
    """
    
    def __shuffle(self):
        X = self.X
        y = self.y
        idx = np.arange(X.shape[0])
        np.random.shuffle(idx)
        self.X = X[idx]
        self.y = y[idx]
        self.require_shuffle = False
        return
    
    def __init__(self, X, y, batch_size = 32, epochs = 10, seed = 1):
        
        if not isinstance(X, np.ndarray) or not isinstance(y, np.ndarray):
            raise ValueError('Both X and y must be np.ndarray')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must be of same size of axis=0")
        
        from math import ceil
        np.random.seed(seed)
        self.X = X
        self.y = y
        self.epochs = epochs
        self.batch_size = batch_size
        self.num_batches = ceil(X.shape[0] / batch_size)
        self.max_iters = self.epochs * self.num_batches
        self.__shuffle()
            
        
    def next(self):
        from time import time
        start_time = time()
        iteration, progress, rate = 0, 0, 0
        for i in range(self.epochs):
            self.current_epoch = i
            for j in range(self.num_batches):
                self.current_batch = j
                start = j * self.batch_size
                end = start + self.batch_size
                iteration = iteration + 1
                progress = int(100 * iteration / self.max_iters) + 1
                rate = (time() - start_time) / iteration
                info = BatchableInfo()
                info.iteration = iteration
                info.progress = progress
                info.rate = rate
                info.show_progress = iteration % (self.max_iters//20) == 0 or iteration == self.max_iters - 1 
                yield self.X[start:end], self.y[start:end], info
                
            self.__shuffle()

In [148]:
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

max_iters = 5000

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


init = tf.global_variables_initializer()
batchable = Batchable(X_train, y_train, epochs=10)
with tf.Session() as sess:
    init.run()
    for X_batch, y_batch, info in batchable.next():
        sess.run(opt, feed_dict={X: X_batch, y: y_batch})
        if info.show_progress:
            acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print("Progress:%3d%%" % info.progress, 
                  "Train accuracy: %.4f" % acc_train, 
                  "Test accuracy: %.4f (ms)" % acc_test,
                  "Rate %.4f" % info.rate)

Progress:  5% Train accuracy: 0.9062 Test accuracy: 0.8694 (ms) Rate 0.0081
Progress: 10% Train accuracy: 0.9062 Test accuracy: 0.9325 (ms) Rate 0.0086
Progress: 15% Train accuracy: 0.9688 Test accuracy: 0.9420 (ms) Rate 0.0098
Progress: 20% Train accuracy: 0.9375 Test accuracy: 0.9406 (ms) Rate 0.0096
Progress: 25% Train accuracy: 0.9688 Test accuracy: 0.9299 (ms) Rate 0.0097
Progress: 30% Train accuracy: 0.9688 Test accuracy: 0.9634 (ms) Rate 0.0098
Progress: 35% Train accuracy: 1.0000 Test accuracy: 0.9539 (ms) Rate 0.0097
Progress: 40% Train accuracy: 0.9688 Test accuracy: 0.9550 (ms) Rate 0.0098
Progress: 45% Train accuracy: 0.9688 Test accuracy: 0.9638 (ms) Rate 0.0101
Progress: 50% Train accuracy: 0.9688 Test accuracy: 0.9556 (ms) Rate 0.0101
Progress: 55% Train accuracy: 0.9375 Test accuracy: 0.9633 (ms) Rate 0.0101
Progress: 60% Train accuracy: 0.9688 Test accuracy: 0.9448 (ms) Rate 0.0100
Progress: 65% Train accuracy: 1.0000 Test accuracy: 0.9556 (ms) Rate 0.0100
Progress: 70